<a href="https://colab.research.google.com/github/gadm21/AI/blob/master/fasttext_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fake and real news classifier


In [1]:
%%capture

! wget https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
! unzip v0.9.2.zip

%cd fastText-0.9.2
! make 
! pip install .

%cd .. 


In [2]:
import pandas as pd 
import numpy as np

def get_dataset():
  !wget --load-cookies /tmp/cookies.txt\
   "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt\
    --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1EzY1IfN_QGCVp9EUVxZ3dZhRF_EUtyJA' -O- \
    | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1EzY1IfN_QGCVp9EUVxZ3dZhRF_EUtyJA" -O kaggle.json && rm -rf /tmp/cookies.txt

  !pip install kaggle

  !mkdir ~/.kaggle
  !cp kaggle.json ~/.kaggle/
  !chmod 600 ~/.kaggle/kaggle.json
  !kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset
  !unzip 'fake-and-real-news-dataset.zip'

def csv_to_txt(csv_path, split_ratio=0.2): 

  train_txt = csv_path.split('.')[0]+ '_train' + '.txt' 
  valid_txt = csv_path.split('.')[0]+ '_valid' + '.txt'
  valid_len = int(pd.read_csv(csv_path).shape[0] * split_ratio)
  with open(train_txt, 'w') as train_txt_file, open(valid_txt, 'w') as valid_txt_file, open(csv_path, 'r') as csv_file :
      for i, row in enumerate(csv_file) :
        if i > valid_len: train_txt_file.write(row)
        else: valid_txt_file.write(row) 

def csv_splitAndMerge(csv_path1, csv_path2, split_ratio=0.2):
  df = pd.read_csv(csv_path1)
  df['label'] = 'true'
  valid_len = int(df.shape[0]*split_ratio)
  valid_df = df.iloc[:valid_len, :] 
  train_df = df.iloc[valid_len:, :] 

  df2 = pd.read_csv(csv_path2)
  df2['label'] = 'fake'
  valid_len2 = int(df2.shape[0]*split_ratio)
  valid_df2 = df2.iloc[:valid_len2, :] 
  train_df2 = df2.iloc[valid_len2:, :] 

  train_data = pd.concat([train_df, train_df2])
  valid_data = pd.concat([valid_df, valid_df2])

  train_data = train_data.iloc[np.random.permutation(len(train_data))]
  valid_data = valid_data.iloc[np.random.permutation(len(valid_data))]
  return train_data, valid_data


def to_trainable(df, txt_path):
  txt_file = open(txt_path, 'w') 

  for i, row in df.iterrows():
    txt_file.write('__label__'+row['label']+' '+row['text']+'\n')


In [3]:
%%capture

get_dataset()
train_data, valid_data  = csv_splitAndMerge('True.csv', 'Fake.csv')

to_trainable(train_data, 'train.txt') 
to_trainable(valid_data, 'valid.txt')

In [4]:
import fasttext 
model = fasttext.train_supervised(input='train.txt') 
model.save_model('news_classifier.bin')

In [6]:
model.test("valid.txt")

(8979, 0.9988862902327653, 0.9988862902327653)

In [16]:
model.predict(' Donald Trump Sends Out Embarrassing New Year’s Eve Message; This is Disturbing')

(('__label__fake',), array([1.00001001]))